In [6]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

# Function to crop the circular region of the fundus image
def crop_fundus_circle(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply a Gaussian blur to remove noise
    blurred = cv2.GaussianBlur(gray, (9, 9), 0)

    # Use Hough Circle Transform to detect the circular region
    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1.2, minDist=1000,
                               param1=50, param2=30, minRadius=400, maxRadius=800)

    if circles is not None:
        # Convert the (x, y, radius) values to integers
        circles = np.round(circles[0, :]).astype("int")
        for (x, y, r) in circles:
            # Create a mask with the same dimensions as the image, filled with zeros
            mask = np.zeros_like(gray)

            # Draw a white filled circle on the mask at the detected circle's position
            cv2.circle(mask, (x, y), r, 255, thickness=-1)

            # Apply the mask to the image to keep only the circular region
            masked_img = cv2.bitwise_and(image, image, mask=mask)

            # Crop the image around the circular region
            x1, y1 = x - r, y - r
            x2, y2 = x + r, y + r
            cropped_img = masked_img[y1:y2, x1:x2]

            return cropped_img

    return image  # Return original image if no circle is detected

# Define the input and output directories
input_dir = 'C:/Users/PC_I/OneDrive/Desktop/IN'
output_dir = 'C:/Users/PC_I/OneDrive/Desktop/out'  # Output folder to save cropped images

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Loop through all files in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith('.png') or filename.endswith('.jpg'):  # Process only image files
        image_path = os.path.join(input_dir, filename)
        
        # Load the image
        img = cv2.imread(image_path)
        
        # Check if the image was loaded successfully
        if img is None:
            print(f"Error: Could not load {filename}. Skipping...")
            continue
        
        # Crop the circular region of the fundus image
        cropped_img = crop_fundus_circle(img)
        
        # Construct the output path using the original filename
        output_path = os.path.join(output_dir, filename)
        
        # Save the cropped image
        cv2.imwrite(output_path, cropped_img)

  

        print(f"Cropped image saved as {output_path}")
    else:
        print(f"Skipping non-image file: {filename}")


Cropped image saved as C:/Users/PC_I/OneDrive/Desktop/out\1492_right.jpg
Cropped image saved as C:/Users/PC_I/OneDrive/Desktop/out\_348_2941106.jpg
Cropped image saved as C:/Users/PC_I/OneDrive/Desktop/out\_356_2280553.jpg
Cropped image saved as C:/Users/PC_I/OneDrive/Desktop/out\_64_3530889.jpg
Cropped image saved as C:/Users/PC_I/OneDrive/Desktop/out\_64_4692255.jpg
Cropped image saved as C:/Users/PC_I/OneDrive/Desktop/out\_71_9012457.jpg


In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

# Function to crop the circular region of the fundus image
def crop_fundus_circle(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (9, 9), 0)

    # Adjust parameters dynamically based on image size
    height, width = gray.shape
    min_radius = int(min(height, width) * 0.2)
    max_radius = int(min(height, width) * 0.4)

    circles = cv2.HoughCircles(
        blurred, cv2.HOUGH_GRADIENT, dp=1.2, minDist=height//2,
        param1=50, param2=30, minRadius=min_radius, maxRadius=max_radius
    )
  
# Creates a circular mask to isolate the detected fundus region.
# Crops a square region around the circle, ensuring it stays within bounds.
# Returns the cropped result for saving.
  
    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        for (x, y, r) in circles:
            mask = np.zeros_like(gray)  
            cv2.circle(mask, (x, y), r, 255, thickness=-1)
            masked_img = cv2.bitwise_and(image, image, mask=mask)

            # Crop around the detected circle
            x1, y1 = max(0, x - r), max(0, y - r)
            x2, y2 = min(width, x + r), min(height, y + r)
            cropped_img = masked_img[y1:y2, x1:x2]

            return cropped_img

    print("No circle detected. Returning original image.")
    return image

# Paths
input_dir = 'C:/Users/PC_I/OneDrive/Desktop/IN'
output_dir = 'C:/Users/PC_I/OneDrive/Desktop/out'
os.makedirs(output_dir, exist_ok=True)

# Process files
total_files = len(os.listdir(input_dir))
processed_files = 0

for filename in os.listdir(input_dir):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(input_dir, filename)
        img = cv2.imread(image_path)

        if img is None:
            print(f"Error loading {filename}. Skipping...")
            continue

        cropped_img = crop_fundus_circle(img)
        output_path = os.path.join(output_dir, filename)
        cv2.imwrite(output_path, cropped_img)
        print(f"Cropped image saved: {output_path}")

        # Progress update
        processed_files += 1
        print(f"Processed {processed_files}/{total_files} ({processed_files / total_files:.2%})")
    else:
        print(f"Skipping non-image file: {filename}")

print("Processing complete!")


Cropped image saved: C:/Users/PC_I/OneDrive/Desktop/out\1492_right.jpg
Processed 1/6 (16.67%)
Cropped image saved: C:/Users/PC_I/OneDrive/Desktop/out\_348_2941106.jpg
Processed 2/6 (33.33%)
Cropped image saved: C:/Users/PC_I/OneDrive/Desktop/out\_356_2280553.jpg
Processed 3/6 (50.00%)
Cropped image saved: C:/Users/PC_I/OneDrive/Desktop/out\_64_3530889.jpg
Processed 4/6 (66.67%)
Cropped image saved: C:/Users/PC_I/OneDrive/Desktop/out\_64_4692255.jpg
Processed 5/6 (83.33%)
No circle detected. Returning original image.
Cropped image saved: C:/Users/PC_I/OneDrive/Desktop/out\_71_9012457.jpg
Processed 6/6 (100.00%)
Processing complete!
